In [27]:
from torch.utils.data import Dataset,DataLoader,TensorDataset
from torchmetrics import SymmetricMeanAbsolutePercentageError
import matplotlib.pyplot as plt
import torch.nn as nn
import pandas as pd
import numpy as np
import torch
import gc

In [28]:
df = pd.read_csv('Data/All_music.csv')
df["timestamp"] = pd.to_datetime(df["timestamp"])
df.drop("Unnamed: 0",inplace=True,axis=1)
df.set_index("timestamp",inplace=True)

df = df["2020-12-14":]
train_set = df['2020-12-14':'2021-12-14']
valid_set = df['2021-12-15':]





df["dayofweek"] = df.index.dayofweek.astype("int32")
df["log_views"] = np.log(df["views"] + 1e-8)
df["month"] = df.index.month.astype("int32")
df["year"] = df.index.year.astype("int32")
df["avg_by_song_month"] = df.groupby(["month","year","article"],observed=True)["views"].transform("mean").astype("float64")
df["min_by_song_month"] = df.groupby(["month","year","article"],observed=True)["views"].transform("min").astype("float64")
df["max_by_song_month"] = df.groupby(["month","year","article"],observed=True)["views"].transform("max").astype("float64")

features = ["dayofweek", "month", "year", "min_by_song_month", "max_by_song_month", "avg_by_song_month","log_views"]
target = "views"

In [29]:
class MusicDataset(Dataset):
    def __init__(self,features,target):
        self.feature = features
        self.target = target
    
    def __len__(self):
        return len(self.feature)
    
    def __getitem__(self,idx):
        item = self.feature[idx]
        label = self.target[idx]
        
        return item,label

In [30]:
from sklearn.model_selection import train_test_split
import pytorch_lightning as pl
from pytorch_lightning.callbacks import EarlyStopping, LearningRateMonitor
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


early_stop_callback = EarlyStopping(monitor="val_loss", min_delta=1e-4, patience=10, verbose=False, mode="min")
lr_logger = LearningRateMonitor()  # log the learning rate

trainer = pl.Trainer(
    max_epochs=100,
    accelerator='gpu', 
    devices=1,
    gradient_clip_val=0.1,
    #limit_train_batches=30,  # coment in for training, running valiation every 30 batches
    limit_train_batches=1.0, #if set to 1.0 gather all training data, default.
    callbacks=[lr_logger, early_stop_callback],
)

train, test = train_test_split(df, test_size=0.2, shuffle=True)



train_x = torch.tensor(train[features].values).to(device)
train_y = torch.tensor(train[target].values).unsqueeze(1).to(device)
test_x = torch.tensor(test[features].values).to(device)
test_y = torch.tensor(test[target].values).unsqueeze(1).to(device)

train = MusicDataset(train_x,train_y)
valid = MusicDataset(test_x,test_y)

train_loader = DataLoader(train,shuffle=False)
valid_loader = DataLoader(valid,shuffle=False)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
`Trainer(limit_train_batches=1.0)` was configured so 100% of the batches per epoch will be used..


In [74]:
# find optimal learning rate

class NeuralNetwork(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(NeuralNetwork, self).__init__()
        self.layer_1 = nn.Linear(input_dim, hidden_dim)
        nn.init.kaiming_uniform_(self.layer_1.weight, nonlinearity="relu")
        self.layer_2 = nn.Linear(hidden_dim, output_dim)
       
    def forward(self, x):
        x = torch.nn.functional.relu(self.layer_1(x))
        x = torch.nn.functional.sigmoid(self.layer_2(x))

        return x


def smape(target, forecast):
  if type(target) == pd.core.frame.DataFrame:
    target = target.values

  denominator = np.abs(target) + np.abs(forecast)
  flag = denominator == 0.

  smape = 2 * np.mean(
      (np.abs(target - forecast) * (1 - flag)) / (denominator + flag)
  )
  return smape

In [79]:
# train the model
def train_model(train_dl, model):
    # define the optimization
    criterion = nn.MSELoss().to(device)
    optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9, nesterov=True)
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=4, gamma=0.1)
    # enumerate epochs
    for epoch in range(100):
        # enumerate mini batches
        for i, (inputs, targets) in enumerate(train_dl):
            #Decay Learning Rate
            # clear the gradients
            optimizer.zero_grad()
            # compute the model output
            yhat = model(inputs.float())
            # calculate loss
            loss = criterion(yhat.float(), targets.float())
            # credit assignment
            loss.backward()

            # update model weights
            optimizer.step()

            for name, param in model.named_parameters():
                print(name, param.grad)

            scheduler.step()
            print(f"epoch: {epoch}, Loss: {loss}, LR: {scheduler.get_last_lr()}")


# evaluate the model
def evaluate_model(test_dl, model):
    predictions, actuals = list(), list()
    for i, (inputs, targets) in enumerate(test_dl):
        # evaluate the model on the test set
        yhat = model(inputs.float())
        # retrieve numpy array
        yhat = yhat.cpu().detach().numpy()
        actual = targets.numpy()
        actual = actual.reshape((len(actual), 1))
        # round to class values
        yhat = yhat.round()
        # store
        predictions.append(yhat)
        actuals.append(actual)
    predictions, actuals = np.vstack(predictions), np.vstack(actuals)
    # calculate accuracy
    return smape(actuals, predictions)


# make a class prediction for one row of data
def predict(row, model):
    # convert row to data
    row = torch.Tensor([row])
    # make prediction
    yhat = model(row)
    # retrieve numpy array
    yhat = yhat.detach().numpy()
    return yhat

n_input = len(features)
n_hidden = 15
batch_size = 100
learning_rate = 0.01

model = NeuralNetwork(n_input,n_hidden,1).to(device)
train_model(train_loader,model)
# model
# for x,y in train_loader:
#     print(x,y)
#     break

model

layer_1.weight tensor([[0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [-0., -0., -0., -0., -0., -0., -0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [-0., -0., -0., -0., -0., -0., -0.],
        [-0., -0., -0., -0., -0., -0., -0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.]], device='cuda:0')
layer_1.bias tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       device='cuda:0')
layer_2.weight tensor([[-0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0.]],
       device='cuda:0')
layer_2.bias tensor([0.], device='cuda:0')
epoch: 0, Loss: 1357225.0, LR: [0.01]
layer_1.weight tensor([[0., 0., 0., 0., 0., 0., 0.],
        [0.

KeyboardInterrupt: 